## Import libs and custom vars

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from libs.utils import find_penalty, return_outcome

<ul>
    <li>Ci sono alcuni nomi di squadre che sono cambiate nel tempo, uniformizza </li>
    <li>Shootouts è il db dei penalty, quindi bisogna manipolare il dataset per vedere chi vince alla fine quella partita</li>
</ul>

## Descrizione del dataset e prima manipolazione

Il Dataset contiene 42483 risultati di partite di calcio internazionale,

Il primo match risale al 1872-11-30 mentre l’ultimo è del 2021-08-01

Nel dataset sono incluse sia le partite ufficiali di tutte le competizioni, dalle amichevoli alla coppa del mondo. è da specificare inoltre che comprende solo le competizioni nazionali maschili e sono esclusi: Olympic Games or matches where at least one of the teams was the nation's B-team, U-23 or a league select team.

Inoltre, il dataset presenta un ulteriore file che contiene, in totale 432 risultati finali di partite terminate ai calci di rigore (vincenti).

Per correttezza, controlleremo inizialmente che il Dataset non contenga elementi nulli e i due dataset verranno inoltre ‘mergiate’ in un unico file contenti i risultati, dove presenti, anche dopo i calci di rigore.

In [2]:
data = {
    'results': pd.read_csv('dataset/results.csv'),
    'shootouts': pd.read_csv('dataset/shootouts.csv')
}

# print('**'*5, ' results.csv ', '**'*5)
# print(data['results'].info())
# print('**'*5, ' shootouts.csv ', '**'*5)
# print(data['shootouts'].info())

Il dataset non contiene elementi nulli. Visualizziamo le prime 5 righe di ogni dataset

In [ ]:
print('**'*5, ' results.csv ', '**'*5)
print(data['results'].head(5))
print('**'*5, ' shootouts.csv ', '**'*5)
print(data['shootouts'].head(5))

Applichiamo dunque alcune modifiche al dataset. Si integrano di conseguenza alcune informazioni del dataset 'shootouts'

In particolare, processeremo il dataset (partendo da quello di results.csv):
- aggiungendo la colonna 'outcome', sarà classificato come:
    - H (home win)
    - A (away win)
    - D (draw), in particolare se in 'shootouts' è presente la partita 'draw':
        - HP (home win at penalty)
        - AP (away win at penalty)

- saranno estratti i dati relativi alla data della partita in quanto serviranno successivamente

In [4]:
dst_df = data['results'].copy()

res_len = data['results'].__len__()
sht_len = data['shootouts'].__len__()

print('results.csv tot records: %d | shape: %s' % (res_len, data['results'].shape))
print('shootouts.csv tot records: %d | shape: %s' % (sht_len, data['shootouts'].shape))

results.csv tot records: 42483 | shape: (42483, 9)
shootouts.csv tot records: 432 | shape: (432, 4)


In [5]:
# aggiungi una colonna al dataFrame che aggiorna con i risultati dei rigori

dst_df['outcome'] = 'NA'
dst_df['day_of_week'] = 'NA'
dst_df['month'] = 'NA'
dst_df['year'] = 'NA'

counter = 0
for i in range(0, res_len - 1):
    counter = counter + 1
    print('*** Processing %d/%d ***' % (counter, res_len), end='\r')

    # estraggo dati necessari dalla data
    row_date = datetime.fromisoformat(dst_df.iloc[i]['date'])

    dst_df.at[i, 'day_of_week'] = row_date.strftime('%A')
    dst_df.at[i, 'month'] = row_date.strftime('%B')
    dst_df.at[i, 'year'] = int(row_date.strftime('%Y'))

    # calcolo il risultato della partita
    dst_df.at[i, 'outcome'] = return_outcome(data['results'].iloc[i]['home_score'], data['results'].iloc[i]['away_score'])

    if(dst_df.iloc[i]['outcome'] == 'D'):
        dst_df.at[i, 'outcome'] = find_penalty(data['results'].iloc[i], data['shootouts'], sht_len - 1)


dst_df.to_csv('dataset/dataset.csv')

In [6]:
print(dst_df.head())

         date home_team away_team  home_score  away_score tournament     city  \
0  1872-11-30  Scotland   England           0           0   Friendly  Glasgow   
1  1873-03-08   England  Scotland           4           2   Friendly   London   
2  1874-03-07  Scotland   England           2           1   Friendly  Glasgow   
3  1875-03-06   England  Scotland           2           2   Friendly   London   
4  1876-03-04  Scotland   England           3           0   Friendly  Glasgow   

    country  neutral outcome day_of_week     month  year  
0  Scotland    False       D    Saturday  November  1872  
1   England    False       H    Saturday     March  1873  
2  Scotland    False       H    Saturday     March  1874  
3   England    False       D    Saturday     March  1875  
4  Scotland    False       H    Saturday     March  1876  


## Data visualization

importiamo nuovamente dal nuovo csv il dataset manipolato

In [29]:
dst_df = pd.read_csv('dataset/dataset.csv')
dst_df.rename(columns = {'Unnamed: 0':'match_id'}, inplace = True)

print('tot records: %d | shape: %s' % (dst_df.__len__(), dst_df.shape))

tot records: 42483 | shape: (42483, 14)


In [27]:
dst_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42483 entries, 0 to 42482
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   match_id     42483 non-null  int64  
 1   date         42483 non-null  object 
 2   home_team    42483 non-null  object 
 3   away_team    42483 non-null  object 
 4   home_score   42483 non-null  int64  
 5   away_score   42483 non-null  int64  
 6   tournament   42483 non-null  object 
 7   city         42483 non-null  object 
 8   country      42483 non-null  object 
 9   neutral      42483 non-null  bool   
 10  outcome      42482 non-null  object 
 11  day_of_week  42482 non-null  object 
 12  month        42482 non-null  object 
 13  year         42482 non-null  float64
dtypes: bool(1), float64(1), int64(3), object(9)
memory usage: 4.3+ MB


Controlliamo la distribuzione delle partite negli anni per suddividerle nelle varie 'epoche calcistiche'